In [2]:
!pip install --q git+https://github.com/m-bain/whisperx.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 14.2 MB/s eta 0:

In [2]:
import os
from scipy.ndimage import gaussian_filter1d
import librosa
import soundfile as sf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import whisperx
import gc
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast





#dependencies
def extract_features(audio_file, sample_rate=22050, mfcc=True, chroma=True, mel=True):
    audio, sample_rate = librosa.load(audio_file, sr=sample_rate)
    features = []
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
        features.append(mfccs)
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)
        features.append(chroma)
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate).T, axis=0)
        features.append(mel)
    return np.concatenate(features)



def predict_audio(audio_file):
    features = extract_features(audio_file)
    features = np.expand_dims(features, axis=0)
    prediction = model.predict(features)
    if prediction == 0:
        return "not fear - 0"
    elif prediction > 0 and prediction <= 0.1:
        return "fear - 1"
    elif prediction > 0.1 and prediction <= 0.2:
        return "fear - 2"
    elif prediction > 0.2 and prediction <= 0.3:
        return "fear - 3"
    elif prediction > 0.3 and prediction <= 0.4:
        return "fear - 4"
    elif prediction > 0.4 and prediction <= 0.5:
        return "fear - 5"
    elif prediction > 0.5 and prediction < 0.6:
        return "fear - 6"
    elif prediction > 0.6 and prediction < 0.7:
        return "fear - 7"
    elif prediction > 0.7 and prediction < 0.8:
        return "fear - 8"
    elif prediction > 0.8 and prediction < 0.9:
        return "fear - 9"
    elif prediction > 0.9 and prediction < 1:
        return "fear - 10"

#components
def preprosessor(noisy_audio,sr):
    denoised_audio = gaussian_filter1d(noisy_audio, sigma=2)
    sf.write('denoised_audio_hardlevel.mp3', denoised_audio, sr)

def sentiment_score(audio_file_path):
    prediction = predict_audio(audio_file_path)
    return prediction

def annotate(audio_file,senti):
    senti_score=senti
    device = "cpu"
    audio_file = "audio.mp3"
    batch_size = 2
    compute_type = "int8"
    # 1. Transcribe with original whisper (batched)
    model = whisperx.load_model("large-v2", device, compute_type=compute_type)
    audio = whisperx.load_audio(audio_file)
    result = model.transcribe(audio, batch_size=batch_size)
    # 2. Align whisper output
    model_a, metadata = whisperx.load_align_model(model_name="Harveenchadha/vakyansh-wav2vec2-tamil-tam-250",language_code=result["language"], device=device)
    result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)
    # 3. Assign speaker labels
    diarize_model = whisperx.DiarizationPipeline(use_auth_token=HF_TOKEN, device=device) #REPLACE HF_TOKEN WITH YOUR HUGGINGFACE TOKEN
    diarize_segments = diarize_model(audio)
    result = whisperx.assign_word_speakers(diarize_segments, result)
    combined_dialogue = ''
    current_speaker = None
    current_dialogue = ''
    for segment in result['segments']:
        speaker = segment['speaker']
        text = segment['text']
        words = segment['words']
        dialogue = ' '.join([word['word'] for word in words])

        if current_speaker == speaker:
            current_dialogue += ' ' + dialogue
        else:
            if current_speaker is not None:
                combined_dialogue += f"{current_speaker}: {current_dialogue}\n"
            current_speaker = speaker
            current_dialogue = dialogue
    if current_speaker is not None:
        combined_dialogue += f"{current_speaker}: {current_dialogue}"

    translator(combined_dialogue,senti_score)

def translator(text, senti):
    source_lang="ta_IN"
    target_lang="en_XX"
    model_name = "facebook/mbart-large-50-many-to-one-mmt"
    model = MBartForConditionalGeneration.from_pretrained(model_name)
    tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[source_lang], forced_eos_token_id=tokenizer.lang_code_to_id[target_lang])
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    text_sentiment(translated_text,senti)

def text_sentiment(text,audio_senti):
    # Load sentiment analysis pipeline
    sentiment_analysis = pipeline('sentiment-analysis')
    # Example usage
    result = sentiment_analysis(text)
    # Print sentiment and confidence score
    sentiment = result[0]['label']
    confidence_score = result[0]['score']
    summarizer(text,sentiment, confidence_score,audio_senti)


def summarizer(text,sentiment,confidence_score,audio_senti):
    classifier = pipeline("summarization")
    summarized_text=classifier(f"{text}",max_length=56)
    result = f"TEXT-SENTIMENT: {sentiment}\nCONFIDENCE-SCORE: {confidence_score}\nAUDIO-SENTIMENT-SCORE: {audio_senti}\nSUMMARIZED-TEXT: {summarized_text}"
    print(result)


#preprocessing
noisy_audio, sr = librosa.load(r'audio.mp3', sr=None)
preprosessor(noisy_audio,sr)
#sentiment analysis
model = load_model("fear_detection_model3.h5")
audio_path="denoised_audio_hardlevel.mp3"
senti_score=sentiment_score(audio_path)
#print(senti_score)
#speaker segmentation
annotate(audio_path,senti_score)

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


1/1 [==============================] - 0s 218ms/step
No language specified, language will be first be detected for each audio file (increases inference time).


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.0+cu121. Bad things might happen unless you revert torch to 1.x.
Detected language: ta (0.99) in first 30s of audio...


preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

...

TEXT-SENTIMENT: NEGATIVE
CONFIDENCE-SCORE: 0.9921931624412537
AUDIO-SENTIMENT-SCORE: fear - 1
SUMMARIZED-TEXT: [{'summary_text': " Do something like contacting your father to find out who you are . Do what you like. But one condition. You only have one year. Do what it like, but one condition, is contacting your dad . You don't understand now sir. Do it now."}]
1/1 [==============================] - 0s 78ms/step
No language specified, language will be first be detected for each audio file (increases inference time).


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.0+cu121. Bad things might happen unless you revert torch to 1.x.
Detected language: ta (0.99) in first 30s of audio...


Some weights of the model checkpoint at Harveenchadha/vakyansh-wav2vec2-tamil-tam-250 were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Harveenchadha/vakyansh-wav2vec2-tamil-tam-250 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
Y

TEXT-SENTIMENT: NEGATIVE
CONFIDENCE-SCORE: 0.9921931624412537
AUDIO-SENTIMENT-SCORE: fear - 1
SUMMARIZED-TEXT: [{'summary_text': " Do something like contacting your father to find out who you are . Do what you like. But one condition. You only have one year. Do what it like, but one condition, is contacting your dad . You don't understand now sir. Do it now."}]
